In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('Restaurant reviews.csv')

In [3]:
df.head()

,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [4]:
df.shape

(10000, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Restaurant  10000 non-null  object 
 1   Reviewer    9962 non-null   object 
 2   Review      9955 non-null   object 
 3   Rating      9962 non-null   object 
 4   Metadata    9962 non-null   object 
 5   Time        9962 non-null   object 
 6   Pictures    10000 non-null  int64  
 7   7514        1 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [6]:
null_percentage = (df.isnull().mean() * 100).round(2)
null_percentage

Restaurant     0.00
Reviewer       0.38
Review         0.45
Rating         0.38
Metadata       0.38
Time           0.38
Pictures       0.00
7514          99.99
dtype: float64

In [7]:
# drop unneccessary columns
df.drop(columns=['Reviewer', '7514'],axis=1,inplace=True)

In [8]:
df[df.isnull().any(axis=1)]

,Restaurant,Review,Rating,Metadata,Time,Pictures
2360,Amul,NaN,5,0 Reviews,7/29/2018 18:00,0
5799,Being Hungry,NaN,5,"4 Reviews , 4 Followers",7/19/2018 23:55,0
6449,Hyderabad Chefs,NaN,5,1 Review,7/23/2018 16:29,0
6489,Hyderabad Chefs,NaN,5,1 Review,7/8/2018 21:19,0
7954,Olive Garden,NaN,3,"1 Review , 1 Follower",8/9/2018 23:25,0
8228,Al Saba Restaurant,NaN,5,1 Review,7/20/2018 22:42,0
8777,American Wild Wings,NaN,NaN,NaN,NaN,0
8778,American Wild Wings,NaN,NaN,NaN,NaN,0
8779,American Wild Wings,NaN,NaN,NaN,NaN,0
8780,American Wild Wings,NaN,NaN,NaN,NaN,0


In [9]:
df.dropna(inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9955 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Restaurant  9955 non-null   object
 1   Review      9955 non-null   object
 2   Rating      9955 non-null   object
 3   Metadata    9955 non-null   object
 4   Time        9955 non-null   object
 5   Pictures    9955 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 544.4+ KB


In [12]:
# extract data from Metadata column
df[['Reviews', 'Followers']] = df['Metadata'].str.extract(r'(\d+) Reviews.*?(\d+) Followers')


In [17]:
df[df[['Reviews', 'Followers']].isnull().any(axis=1)]

,Restaurant,Review,Rating,Metadata,Time,Pictures,Reviews,Followers
0,Beyond Flavours,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,NaN,NaN
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN,NaN
5,Beyond Flavours,"Ambiance is good, service is good, food is aPr...",5,1 Review,5/24/2019 15:22,0,NaN,NaN
6,Beyond Flavours,"Its a very nice place, ambience is different, ...",5,1 Review,5/24/2019 1:02,0,NaN,NaN
7,Beyond Flavours,Well after reading so many reviews finally vis...,4,1 Review,5/23/2019 15:01,0,NaN,NaN
...,...,...,...,...,...,...,...,...
9948,Chinese Pavilion,Some restaurant's have figured their identity....,5,3 Reviews,10/1/2017 5:21,0,NaN,NaN
9964,Chinese Pavilion,Good ambience and presentation. Thai flavor is...,4,"2 Reviews , 1 Follower",3/26/2017 21:53,0,NaN,NaN
9966,Chinese Pavilion,One of the finest in chinese restaurants. This...,4.5,"1 Review , 56 Followers",2/26/2017 0:57,0,NaN,NaN
9981,Chinese Pavilion,Chinese Cuisine including diverse styles from ...,4.5,"88 Posts , 1694 Followers",10/11/2016 10:10,15,NaN,NaN


In [21]:
df['Metadata'].nunique()

2477